<a href="https://colab.research.google.com/github/shaunck96/Text-To-Speech-Generation-/blob/main/Customer_Service_Audio_Training_Data_Generation_Using_Text_To_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pip
!pip install --upgrade transformers sentencepiece datasets[audio]
pip install --upgrade tensorflow


In [4]:
import torch

In [7]:
from transformers import pipeline
from datasets import load_dataset
import soundfile as sf
import numpy as np
import torch

# Initialize the text-to-speech pipeline with your chosen model.
synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

# Load the dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

# Load speaker embeddings for the customer and the agent
customer_embedding_index = 7306  # Customer's embedding
agent_embedding_index = 1000    # Agent's embedding

customer_speaker_embedding = torch.tensor(embeddings_dataset[customer_embedding_index]["xvector"]).unsqueeze(0)
agent_speaker_embedding = torch.tensor(embeddings_dataset[agent_embedding_index]["xvector"]).unsqueeze(0)

# Define the conversation
conversation = [
    ("Customer", "I believe there's a mistake in my meter reading. My bill is much higher than usual. Can you check it for me?"),
    ("Agent", "Of course, I can help with that. May I have your account number, please?"),
    ("Customer", "It’s 789456123. The reading seems way off compared to what I usually see."),
    ("Agent", "Thank you. Let me pull up your account... It looks like the reading was estimated this month due to an issue accessing your meter. I suggest we schedule a new meter reading. How does that sound?"),
    ("Customer", "That would be great. How soon can someone come out?"),
    ("Agent", "We can have someone there within the next 48 hours. I'll schedule that for you now. We'll adjust your bill based on the new reading."),
    ("Customer", "Perfect, thank you for your help."),
    ("Agent", "No problem at all. If you have any more questions or need further assistance after the reading, feel free to contact us.")
]

# List to store audio segments
audio_segments = []

for speaker, text in conversation:
    # Choose the appropriate speaker embedding
    speaker_embedding = customer_speaker_embedding if speaker == "Customer" else agent_speaker_embedding

    # Generate speech for each segment
    speech = synthesiser(text, forward_params={"speaker_embeddings": speaker_embedding})
    audio_segments.append(speech["audio"])

# Merge audio segments
merged_audio = np.concatenate(audio_segments)

# Write the merged audio to a file
sf.write("meter_reading_conversation.wav", merged_audio, samplerate=speech["sampling_rate"])
